In [99]:
import pandas as pd
import os
import random
import time
import seaborn as sns
import matplotlib.pyplot as plt

In [124]:
data_dir = '/opt/ml/input/data/'
csv_file_path = os.path.join(data_dir, 'train_data_tag_fe.csv')
df = pd.read_csv(csv_file_path)
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,year,month,day,hour,...,test_type_lvl_cat,test_total_answer,tag_ans_mean,tag_ans_sum,tag_ans_std,tag_time_mean,tag_time_std,tag_lvl,tag_lvl_cat,tag_total_answer
0,0,A060001001,A060000001,1,1585009031,7224,2020,3,24,0,...,1,0,0.942913,479,0.232237,15.587706,32.699735,16.531429,0,0
1,0,A060001002,A060000001,1,1585009034,7225,2020,3,24,0,...,1,1,0.913228,3010,0.281543,29.133073,41.916419,31.901199,0,0
2,0,A060001003,A060000001,1,1585009042,7225,2020,3,24,0,...,1,2,0.913228,3010,0.281543,29.133073,41.916419,31.901199,0,1
3,0,A060001004,A060000001,1,1585009049,7225,2020,3,24,0,...,1,3,0.913228,3010,0.281543,29.133073,41.916419,31.901199,0,2
4,0,A060001005,A060000001,1,1585009056,7225,2020,3,24,0,...,1,4,0.913228,3010,0.281543,29.133073,41.916419,31.901199,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,1591339821,438,2020,6,5,6,...,0,1389,0.690377,2805,0.462395,50.524265,59.503996,73.183632,4,4079
2266582,7441,A040165001,A040000165,1,1597971999,8836,2020,8,21,1,...,1,1092,0.699392,2187,0.458596,43.622342,60.170574,62.371771,3,3147
2266583,7441,A040165002,A040000165,1,1597972010,8836,2020,8,21,1,...,1,1093,0.699392,2187,0.458596,43.622342,60.170574,62.371771,3,3148
2266584,7441,A040165003,A040000165,1,1597972056,8836,2020,8,21,1,...,1,1094,0.699392,2187,0.458596,43.622342,60.170574,62.371771,3,3149


In [125]:
def user_fe(df):
    #유저들의 문제 풀이수, 정답 수, 정답률을 시간순으로 누적해서 계산
    df['user_ans_1'] = df[df['userID'].shift(1) == df['userID']].groupby('userID')['answerCode'].transform(lambda x:x.cumsum().shift(1))
    df['user_total_ans'] = df.groupby('userID')['answerCode'].cumcount()
    df['user_acc'] = df['user_ans_1']/df['user_total_ans']

    df['user_test_ans_count'] = df.groupby(['userID','testId'])['answerCode'].cumcount()
    df['user_test_ans_1'] = df[df['userID'].shift(1) == df['userID']].groupby(['userID','testId'])['answerCode'].transform(lambda x:x.cumsum().shift(1))
    df['user_test_acc'] = df['user_test_ans_1'] / df['user_test_ans_count']
    df['user_test_lvl_mean'] = df.groupby(['userID'])['test_lvl'].cumsum() / (df.user_total_ans + 1)

    df['user_tag_ans_count'] = df.groupby(['userID','KnowledgeTag'])['answerCode'].cumcount()
    df['user_tag_ans_1'] = df[df['userID'].shift(1) == df['userID']].groupby(['userID','KnowledgeTag'])['answerCode'].transform(lambda x:x.cumsum().shift(1))
    df['user_tag_acc'] = df['user_tag_ans_1'] / df['user_tag_ans_count']
    df['user_tag_lvl_mean'] = df.groupby(['userID'])['tag_lvl'].cumsum() / (df.user_total_ans + 1)

    df['user_lvl'] = df.user_tag_lvl_mean + df.user_test_lvl_mean
    df = df.fillna(0)
    return df

In [126]:
df = user_fe(df)

In [127]:
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,year,month,day,hour,...,user_acc,user_test_ans_count,user_test_ans_1,user_test_acc,user_test_lvl_mean,user_tag_ans_count,user_tag_ans_1,user_tag_acc,user_tag_lvl_mean,user_lvl
0,0,A060001001,A060000001,1,1585009031,7224,2020,3,24,0,...,NaN,0,NaN,NaN,23.877991,0,NaN,NaN,16.531429,40.409421
1,0,A060001002,A060000001,1,1585009034,7225,2020,3,24,0,...,NaN,1,NaN,NaN,23.877991,0,NaN,NaN,24.216314,48.094306
2,0,A060001003,A060000001,1,1585009042,7225,2020,3,24,0,...,0.500000,2,1.0,0.500000,23.877991,1,1.0,1.00,26.777942,50.655934
3,0,A060001004,A060000001,1,1585009049,7225,2020,3,24,0,...,0.666667,3,2.0,0.666667,23.877991,2,2.0,1.00,28.058757,51.936748
4,0,A060001005,A060000001,1,1585009056,7225,2020,3,24,0,...,0.750000,4,3.0,0.750000,23.877991,3,3.0,1.00,28.827245,52.705236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,1591339821,438,2020,6,5,6,...,0.250000,4,1.0,0.250000,76.231909,4,1.0,0.25,73.183632,149.415540
2266582,7441,A040165001,A040000165,1,1597971999,8836,2020,8,21,1,...,0.200000,0,NaN,NaN,75.435030,0,NaN,NaN,71.381655,146.816686
2266583,7441,A040165002,A040000165,1,1597972010,8836,2020,8,21,1,...,0.333333,1,1.0,1.000000,74.865832,1,1.0,1.00,70.094529,144.960361
2266584,7441,A040165003,A040000165,1,1597972056,8836,2020,8,21,1,...,0.428571,2,2.0,1.000000,74.438933,2,2.0,1.00,69.129184,143.568117


In [ ]:
df
df.to_csv('/opt/ml/input/data/train_data_user_fe.csv',index=False)